In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Install tmdbsimple
!pip install tqdm

In [3]:
with open('/Users/qfu88/.secret/yelp_api.json') as f: 
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
# set our API call parameters 
LOCATION = 'Chicago, IL'
TERM = 'Sandwich'

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Chicago_Sandwich.json"
JSON_FILE

'Data/results_in_progress_Chicago_Sandwich.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_Chicago_Sandwich.json already exists.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 20 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

9500

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

474

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2)

  0%|          | 0/474 [00:00<?, ?it/s]

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/474 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,xHdy8fbDVB4Nb_WGBN50Ow,humboldt-haus-sandwich-bar-chicago-2,Humboldt Haus Sandwich Bar,https://s3-media2.fl.yelpcdn.com/bphoto/efcW7c...,False,https://www.yelp.com/biz/humboldt-haus-sandwic...,253,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 41.910374, 'longitude': -87.701357}","[delivery, pickup]",$,"{'address1': '2958 W North Ave', 'address2': N...",+17736979590,(773) 697-9590,2036.619737
1,JIyH_X4bfwlcYaPZwJMt8g,tribecca-s-sandwich-shop-chicago,TriBecca’s Sandwich Shop,https://s3-media4.fl.yelpcdn.com/bphoto/0nKQrm...,False,https://www.yelp.com/biz/tribecca-s-sandwich-s...,33,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",4.5,"{'latitude': 41.939274964805755, 'longitude': ...",[],NaN,"{'address1': '2949 W Belmont Ave', 'address2':...",+17738782717,(773) 878-2717,4300.250063
2,NmynDaHorWl1uO2ppOouUA,la-sandwichera-chicago,La Sandwichera,https://s3-media2.fl.yelpcdn.com/bphoto/-QPZTK...,False,https://www.yelp.com/biz/la-sandwichera-chicag...,12,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",4.5,"{'latitude': 41.902763142599, 'longitude': -87...",[delivery],NaN,"{'address1': '2721 W Division St', 'address2':...",+17736977477,(773) 697-7477,1479.029452
3,aWGYz1HiFcWBzEtyofewOA,zenwich-x-elitea-wicker-park-chicago-5,Zenwich X Elitea - Wicker Park,https://s3-media3.fl.yelpcdn.com/bphoto/3adpFs...,False,https://www.yelp.com/biz/zenwich-x-elitea-wick...,67,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 41.90772, 'longitude': -87.67291}","[delivery, pickup]",$$,"{'address1': '1415 N Milwaukee Av', 'address2'...",+17736619453,(773) 661-9453,493.424885
4,TrHpyyurfCHWUQMpsLyxVw,bari-foods-chicago-3,Bari Foods,https://s3-media3.fl.yelpcdn.com/bphoto/h23U_D...,False,https://www.yelp.com/biz/bari-foods-chicago-3?...,827,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.5,"{'latitude': 41.891311, 'longitude': -87.65554}","[delivery, pickup]",$,"{'address1': '1120 W Grand Ave', 'address2': N...",+13126660730,(312) 666-0730,2399.567084


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,CF4W2FqdND7l8ofXS6nmMg,tropical-smoothie-cafe-burbank,Tropical Smoothie Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Zb-Aiv...,False,https://www.yelp.com/biz/tropical-smoothie-caf...,2,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",5.0,"{'latitude': 41.751365, 'longitude': -87.7416178}","[pickup, delivery]",NaN,"{'address1': '7784 S Cicero Ave', 'address2': ...",+17085293946,(708) 529-3946,17905.788918
996,EdKG2DIWoHR2W7gc49N4rg,mr-submarine-chicago-4,Mr Submarine,https://s3-media2.fl.yelpcdn.com/bphoto/tYSfOH...,False,https://www.yelp.com/biz/mr-submarine-chicago-...,56,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 41.9249099, 'longitude': -87.66665}","[pickup, delivery]",$,"{'address1': '1525 W Fullerton Ave', 'address2...",+17733486018,(773) 348-6018,2370.724676
997,7cXjpAbeA7ApvKHx6FcCOQ,mcalisters-deli-glenview,McAlister's Deli,https://s3-media3.fl.yelpcdn.com/bphoto/ojhVia...,False,https://www.yelp.com/biz/mcalisters-deli-glenv...,5,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",5.0,"{'latitude': 42.10500897972252, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '2351 Willow Rd', 'address2': '',...",+12245295902,(224) 529-5902,24915.127805
998,B9GC0mRtbXwR4U9fgehMCg,fullerton-restaurant-chicago-2,Fullerton Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/XgY_YD...,False,https://www.yelp.com/biz/fullerton-restaurant-...,85,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",4.0,"{'latitude': 41.92567, 'longitude': -87.687896}",[],$,"{'address1': '2400 W Fullerton Ave', 'address2...",+17733845500,(773) 384-5500,2429.868056
999,fLYKOfpvCmy2jQ6YW2FDjQ,the-drunken-bean-chicago-2,The Drunken Bean Coffee & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/zfX3ig...,False,https://www.yelp.com/biz/the-drunken-bean-chic...,105,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"{'latitude': 41.886711, 'longitude': -87.616767}","[pickup, delivery]",$$,"{'address1': '400 E South Water St', 'address2...",+13125498000,(312) 549-8000,5460.309117


In [17]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

30

In [18]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [19]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Chicago_Sandwich.csv.gz', compression='gzip',index=False)